# Notebook for practicing pySpark

resource: https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf

Goal: Complete everything till page 52

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# you can access all the contexts via spark session. No need to have a separate 
# context for spark, sql, streaming, hive etc
spark = SparkSession\
                .builder.appName("My First Spark App")\
                .config("spark.executors.cores", "4")\
                .config("spark.executors.memory", "10g")\
                .getOrCreate()
# help(SparkSession.builder.config)

In [2]:
data = [[1,2,3,"mom"],
       [2,3,4, "dad"],
       [3,4,5,"mistress"],
       [5,6,7, "jon snow"]]

cols = ['A', 'B', 'C', 'Crime']
p_df = pd.DataFrame(data, columns=cols)
s_df = spark.createDataFrame(data, schema=cols)

In [3]:
p_df

,A,B,C,Crime
0,1,2,3,mom
1,2,3,4,dad
2,3,4,5,mistress
3,5,6,7,jon snow


In [4]:
s_df.show()

+---+---+---+--------+
|  A|  B|  C|   Crime|
+---+---+---+--------+
|  1|  2|  3|     mom|
|  2|  3|  4|     dad|
|  3|  4|  5|mistress|
|  5|  6|  7|jon snow|
+---+---+---+--------+



In [5]:
type(s_df), type(p_df)

(pyspark.sql.dataframe.DataFrame, pandas.core.frame.DataFrame)

In [6]:
# how to create an rdd:
rdd1 = spark.sparkContext.parallelize(data)

In [7]:
type(rdd1)

pyspark.rdd.RDD

In [8]:
# This will convert it into a dataframe
type(rdd1.toDF(cols)) 

pyspark.sql.dataframe.DataFrame

In [9]:
rdd1.collect()

[[1, 2, 3, 'mom'],
 [2, 3, 4, 'dad'],
 [3, 4, 5, 'mistress'],
 [5, 6, 7, 'jon snow']]

In [10]:
rdd1

ParallelCollectionRDD[9] at parallelize at PythonRDD.scala:175

###  A test to see what data structures are supported by RDDs

In [13]:
data_diverse = [[[1], [2], [3]],
                   "test"]
rdd_div = spark.sparkContext.parallelize(data_diverse)
rdd_div.collect()

[[[1], [2], [3]], 'test']

In [37]:
s_df.collect()

[Row(A=1, B=2, C=3, Crime='mom'),
 Row(A=2, B=3, C=4, Crime='dad'),
 Row(A=3, B=4, C=5, Crime='mistress'),
 Row(A=5, B=6, C=7, Crime='jon snow')]

In [ ]:
# reading a csv

In [49]:
# gotta add the header =True tag to use a header else it will break
s_csv_df = spark.read.format('csv')\
                                .options(header=True)\
                                .load("/home/shiv/projects/cnk/datasets/phase2/Results/PositiveData_Tourico_14May.csv")

In [53]:
s_csv_df.show(n=3)
# n is no of rows to display

+--------------------+--------------------+------------------------+--------------------+---------------+--------------------+----------------+-------------------+-------------------+-------------------------------+--------------------+------------+-----------+-----------+--------------------------------------+--------------------+--------------------+------------+--------------+--------------------+----------------------+----------------------+--------------------+--------------------+-------------------------+--------+-----------------------+----------+--------------+-----------------+----------+--------------+--------------+-------------------+------------+--------------------+----------------------+---------------------+--------------------+----------------------+-----------------+--------------------+--------------------+------------+-------------------------------+-----------------------------+--------------------+--------------------+--------------------+-------------+----------

In [54]:
s_csv_df.printSchema()

root
 |-- SupplierRoomName: string (nullable = true)
 |-- AccoRoomName: string (nullable = true)
 |-- SupplierCompoundRoomName: string (nullable = true)
 |-- AccoCompoundRoomName: string (nullable = true)
 |-- SimilarityScore: string (nullable = true)
 |-- SupplierRoomCategory: string (nullable = true)
 |-- SupplierRoomView: string (nullable = true)
 |-- SupplierRoomBedType: string (nullable = true)
 |-- SupplierRoomSmoking: string (nullable = true)
 |-- SupplierRoomExtractedAttributes: string (nullable = true)
 |-- AccoRoomCategory: string (nullable = true)
 |-- AccoRoomView: string (nullable = true)
 |-- AccoBedType: string (nullable = true)
 |-- AccoSmoking: string (nullable = true)
 |-- AccommodationSupplierRoomTypeMappingId: string (nullable = true)
 |-- AccommodationId: string (nullable = true)
 |-- SupplierId: string (nullable = true)
 |-- SupplierName: string (nullable = true)
 |-- SupplierRoomId: string (nullable = true)
 |-- SupplierRoomTypeCode: string (nullable = true)
 |--

In [56]:
# renaming a column in pandas and spark
p_df, s_df

(   A  B  C     Crime
 0  1  2  3       mom
 1  2  3  4       dad
 2  3  4  5  mistress
 3  5  6  7  jon snow,
 DataFrame[A: bigint, B: bigint, C: bigint, Crime: string])

In [57]:
mapping = {'A':'RenamedColumn'}
p_df.rename(columns=mapping, inplace = True)
p_df

,RenamedColumn,B,C,Crime
0,1,2,3,mom
1,2,3,4,dad
2,3,4,5,mistress
3,5,6,7,jon snow


In [58]:
renaming = [mapping.get(col, col) for col in s_df.columns]
# s_df.toDF()
renaming

['RenamedColumn', 'B', 'C', 'Crime']

In [69]:
s_df.toDF(*renaming).show()

+-------------+---+---+--------+
|RenamedColumn|  B|  C|   Crime|
+-------------+---+---+--------+
|            1|  2|  3|     mom|
|            2|  3|  4|     dad|
|            3|  4|  5|mistress|
|            5|  6|  7|jon snow|
+-------------+---+---+--------+



In [70]:
# method 2
s_df.withColumnRenamed('B', 'NewNameForB').show()

+---+-----------+---+--------+
|  A|NewNameForB|  C|   Crime|
+---+-----------+---+--------+
|  1|          2|  3|     mom|
|  2|          3|  4|     dad|
|  3|          4|  5|mistress|
|  5|          6|  7|jon snow|
+---+-----------+---+--------+



In [71]:
# dropping columns
p_df.drop(columns=['B'])

,RenamedColumn,C,Crime
0,1,3,mom
1,2,4,dad
2,3,5,mistress
3,5,7,jon snow


In [77]:
s_df.drop(*['B', 'A']).show() # dropping multiple of them

+---+--------+
|  C|   Crime|
+---+--------+
|  3|     mom|
|  4|     dad|
|  5|mistress|
|  7|jon snow|
+---+--------+



In [78]:
# filters in dataframes are same as filters in pandas dataframes

p_df[p_df.Crime == 'mom']

,RenamedColumn,B,C,Crime
0,1,2,3,mom


In [80]:
s_df[s_df.Crime == 'mom'].show()

+---+---+---+-----+
|  A|  B|  C|Crime|
+---+---+---+-----+
|  1|  2|  3|  mom|
+---+---+---+-----+



In [81]:
# lambdas adn withColumns
p_df['C_norm'] = p_df.C/p_df.C.sum()

In [82]:
p_df

,RenamedColumn,B,C,Crime,C_norm
0,1,2,3,mom,0.157895
1,2,3,4,dad,0.210526
2,3,4,5,mistress,0.263158
3,5,6,7,jon snow,0.368421


In [91]:
# column name, lambda
C_sum = s_df.groupBy().agg(F.sum('C')).collect()[0][0]
s_df.withColumn('C_norm', s_df.C/C_sum).show()

+---+---+---+--------+-------------------+
|  A|  B|  C|   Crime|             C_norm|
+---+---+---+--------+-------------------+
|  1|  2|  3|     mom|0.15789473684210525|
|  2|  3|  4|     dad|0.21052631578947367|
|  3|  4|  5|mistress| 0.2631578947368421|
|  5|  6|  7|jon snow| 0.3684210526315789|
+---+---+---+--------+-------------------+



### sql.fucntions contains a lot of standard math functionalities that we can apply on the columns

In [93]:
# concat a column
s_df.withColumn("concated", F.concat('A', 'Crime')).show()

+---+---+---+--------+---------+
|  A|  B|  C|   Crime| concated|
+---+---+---+--------+---------+
|  1|  2|  3|     mom|     1mom|
|  2|  3|  4|     dad|     2dad|
|  3|  4|  5|mistress|3mistress|
|  5|  6|  7|jon snow|5jon snow|
+---+---+---+--------+---------+



In [ ]:
# ranking values is another important functionality in dataframes

